#### 保存

ポイント

```
dollar_volume.percentile_between(90, 92)   
cond3 = df_data_from_start_to_end.std_prevday.apply(np.log) > -5.5
``` 

流動性のポジションが90％-92％で，(52M~80M) 
前日のstdがそこまで大きくない



In [ ]:
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import Q1500US, Q500US
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import AverageDollarVolume,  CustomFactor,RSI
from quantopian.pipeline.classifiers.morningstar import Sector, SuperSector
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data import morningstar

import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt  

import pytz
tz_ny = pytz.timezone('America/New_York')


class FinancialFactor(CustomFactor):
    window_length = 2
    def compute(self, today, assets, out, v): 
        out[:] = v[0]
        
class MarketCap(FinancialFactor):
    inputs = [morningstar.valuation.market_cap]
    def compute(self, today, assets, out, v): 
        out[:] = np.log(v[0])
    
class ROA(FinancialFactor):
    inputs = [morningstar.operation_ratios.roa]
     
class ROE(FinancialFactor):
    inputs = [morningstar.operation_ratios.roe]

class NormalizedBasicEps(FinancialFactor):
    inputs = [morningstar.earnings_report.normalized_basic_eps]

class NetIncomeGrowth(FinancialFactor):
    inputs = [morningstar.operation_ratios.net_income_growth]

class PE(FinancialFactor):
    inputs = [morningstar.valuation_ratios.pe_ratio]

class BookValueYield(FinancialFactor):
    inputs = [morningstar.valuation_ratios.book_value_yield]

class DividendYield(FinancialFactor):
    inputs = [morningstar.valuation_ratios.dividend_yield]

# class ShortName(FinancialFactor):
#     inputs = [morningstar.company_reference.short_name]
#     def compute(self, today, assets, out, v): 
#         out[:] = np.array(v[0])

class PeriodEndingDate(FinancialFactor):
    inputs = [morningstar.financial_statement_filing.period_ending_date]
       
def make_pipeline():
    base_universe = Q500US()
    dollar_volume = AverageDollarVolume(window_length=30)
    #high_dollar_volume = dollar_volume.percentile_between(98, 100)
    high_dollar_volume = dollar_volume.percentile_between(90, 92)    
    sector = Sector()
    rsi = RSI(inputs=[USEquityPricing.close])
    pipe = Pipeline(
        
        columns = {'dollar_volume': dollar_volume,
                   'high_dollar_volume': high_dollar_volume, 
                   'sector': sector,
                   'rsi': rsi, 
                   'market_cap': MarketCap(),
                   'roa': ROA(),
                   'roe': ROE(),
                   'normalized_basic_eps': NormalizedBasicEps(),
                   'net_income_growth': NetIncomeGrowth(),
                   'pe': PE(),
                   'book_value_yield': BookValueYield(),
                   'dividend_yield': DividendYield(),
                   'period_ending_date': PeriodEndingDate(),
        },
        screen = base_universe & high_dollar_volume #& rsi_under_60
    )
    return pipe

def get_prices(pipeline_results, start_date, end_date):
    sids = pipeline_results.ix[end_date].index
    sids = sids.append([symbols(["spy"])])
    pan_1m = get_pricing(sids, start_date=start_date, end_date=end_date, frequency='minute') 
    pan_1m.major_axis = pan_1m.major_axis.tz_convert(tz_ny)
    pan_1m['turnover'] = pan_1m.price * pan_1m.volume
    pan_1m['minute_change'] = pan_1m.price.pct_change()
    pan_1m['price_change_turnover'] = pan_1m.minute_change.abs() * pan_1m.turnover
    return pan_1m 

def spy(pan_1m):
    spygap = pan_1m.price.ix[389:391].pct_change().at_time("09:31").ix[0].ix[symbols("spy")]
    return spygap 

def get_data(pan_1m):
    
    #gap = pan_1m.price.ix[389:391].pct_change().at_time("09:31").ix[0] - spygap # この計算は結構遅い
    gap = pan_1m.price.pct_change().at_time("09:31").ix[-1]
    std_prevday = pan_1m.minute_change.ix[:390].describe().ix["std"]
    std_prevday_1500_1600 = pan_1m.minute_change.ix[329:390].describe().ix["std"]
    price_change_turnover_0931 = pan_1m.price_change_turnover.ix[390]
    price_today_0931 = pan_1m.price.ix[390]
    price_today_0932 = pan_1m.price.ix[391]
    price_today_0940 = pan_1m.price.ix[399]
    price_today_0950 = pan_1m.price.ix[409]
    price_today_1000 = pan_1m.price.ix[419]
    price_today_1010 = pan_1m.price.ix[429]
    price_today_1020 = pan_1m.price.ix[439]
    price_today_1030 = pan_1m.price.ix[449]
    price_today_1040 = pan_1m.price.ix[459]
    price_today_1050 = pan_1m.price.ix[469]
    price_today_1100 = pan_1m.price.ix[479]

    today = pan_1m.major_axis[-1]#.strftime("%Y-%m-%d")
    prevday = pan_1m.major_axis[0]#.strftime("%Y-%m-%d")

    df = pd.DataFrame({'today': today,
                       'prevday':prevday,
                       'gap_today_0931': gap, 
                       'std_prevday':std_prevday,
                       'std_prevday_1500_1600': std_prevday_1500_1600,
                       'price_change_turnover_0931':price_change_turnover_0931,
                       'price_today_0931':price_today_0931,
                       '1m' : price_today_0932 / price_today_0932 - 1, 
                       '10m': price_today_0940 / price_today_0932 - 1, 
                       '20m': price_today_0950 / price_today_0932 - 1, 
                       '30m': price_today_1000 / price_today_0932 - 1, 
                       '40m': price_today_1010 / price_today_0932 - 1, 
                       '50m': price_today_1020 / price_today_0932 - 1, 
                       '60m': price_today_1030 / price_today_0932 - 1, 
                       '70m': price_today_1040 / price_today_0932 - 1, 
                       '80m': price_today_1050 / price_today_0932 - 1, 
                       '90m': price_today_1100 / price_today_0932 - 1, 
                  })
    return df 



In [ ]:
pipeline_results = run_pipeline(make_pipeline(), start_date='2016-1-1', end_date='2016-4-1')


In [ ]:
def get_data_from_start_to_end(pipeline_results):
    dates = pipeline_results.index.get_level_values(0).unique()
    l1 = list()
    l2 = list()
    for i, date in enumerate(dates):
        if i > 0:
            print date.strftime("%Y-%m-%d"), 
            pan_1m = get_prices(pipeline_results, dates[i-1], date)
            df = get_data(pan_1m)
            df_indicator = pipeline_results.ix[date]
            l1.append(pan_1m)
            l2.append(pd.concat([df,df_indicator], axis=1))
                
    df_data_from_start_to_end = pd.concat(l2)
    return df_data_from_start_to_end,l1


In [ ]:
df_data_from_start_to_end, l1 = get_data_from_start_to_end(pipeline_results)

In [ ]:
#df_all.plot.scatter(x = "gap_today_0931", y="40m")
fig = plt.figure()
thred=0.01
cond1 = df_data_from_start_to_end.gap_today_0931 > 0.01
cond2 = df_data_from_start_to_end.gap_today_0931 < -0.01
cond3 = df_data_from_start_to_end.std_prevday.apply(np.log) > -5.5

x = df_data_from_start_to_end[cond3]
im = plt.scatter(x['gap_today_0931'],
                 x['20m'],
                 c=x['std_prevday'].apply(np.log),
 linewidths=0, alpha=1, 
 cmap=cm.winter # ここでカラーマップを指定
 )
fig.colorbar(im)


In [ ]:
x.std_prevday.min()